In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('calories_burn.csv')

In [10]:
df.head()

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,14733363,male,68,190,94,29,105,40.8,231
1,14861698,female,20,166,60,14,94,40.3,66
2,11179863,male,69,179,79,5,88,38.7,26
3,16180408,female,34,179,71,13,100,40.5,71
4,17771927,female,27,154,58,10,81,39.8,35


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   User_ID     15000 non-null  int64  
 1   Gender      15000 non-null  object 
 2   Age         15000 non-null  int64  
 3   Height      15000 non-null  int64  
 4   Weight      15000 non-null  int64  
 5   Duration    15000 non-null  int64  
 6   Heart_Rate  15000 non-null  int64  
 7   Body_Temp   15000 non-null  float64
 8   Calories    15000 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 1.0+ MB


Up till now I have two observations which are

1- I dont need userid column, i can drop it safely without affecting the model later

2- I need to encode the gender column so the Dtype is changed into meaningfull numbers\

I will encode the gender column later after i do some plot analysis.

In [12]:
df.drop('User_ID', axis=1, inplace=True)

In [13]:
df.head()

,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,68,190,94,29,105,40.8,231
1,female,20,166,60,14,94,40.3,66
2,male,69,179,79,5,88,38.7,26
3,female,34,179,71,13,100,40.5,71
4,female,27,154,58,10,81,39.8,35


In [14]:
df.isna().sum()

Gender        0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [15]:
df['Calories'].value_counts()

Calories
7      161
13     142
12     137
11     137
17     136
      ... 
270      1
280      1
289      1
258      1
287      1
Name: count, Length: 277, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
explode = (0,0.1)
fig, ax = plt.subplots(1,2)
ax[0].pie(explode=explode, x=df['Outcome'].value_counts(), labels=['No Diabetes', 'Diabetes'], autopct='%1.1f%%', startangle=90)
ax[1].bar(['No Diabetes', 'Diabetes'], df['Outcome'].value_counts(), color=['blue', 'orange'])
plt.tight_layout()
plt.show()